This will use following master files to create one master links file having following information.
1. cell_tower_id1
2. cell_tower_id2
3. source1_name
4. source2_name
5. device_name_1
6. device_name_2
7. frequency1
8. frequency2
9. frequency_band
10. latitude1
11. longitude1
12. latitude2
13. longitude2
14. district1
15. district2

Resources for this script would be as following
1. link_frequency_csv - Link_Frequency.csv   #frequency of each link
2. sites_list_csv - Dialog_Site_List.csv     #geolocation of the stations
3. device_resource_csv - Device Resource.csv #To get the device name and resource name 
4. root_folder - root folder name where the above csv are stored. If they are in different places set it to ``""`` and give the absolute paths.

Set the paths for above files and script will produce the following file 
- links.csv - contains the details mentioned in first section

**Before using the out put(links.csv) for the next stage, add id_1, id_2 values by hand. An autoincrementing two values are sufficient.**

In [123]:
root_folder = "/home/gayan/Desktop/CMLARE/Data/Master Files/"
link_frequency_csv = "Link_Frequency.csv"
sites_list_csv = "Dialog_Site_List.csv"
device_resource_csv = "Device Resource.csv"
outputFileName = "links.csv"

In [115]:
link_frequency_csv = root_folder+link_frequency_csv
sites_list_csv = root_folder+sites_list_csv
device_resource_csv = root_folder + device_resource_csv

In [116]:
import pandas as pd

In [117]:
#Loading data
link_frequencies_df = pd.read_csv(link_frequency_csv)
sites_locations_df = pd.read_csv(sites_list_csv,index_col="Site ID")
device_resources_df = pd.read_csv(device_resource_csv,index_col="DeviceName")
print(list(link_frequencies_df))
print(list(sites_locations_df))
print(list(device_resources_df))
links =pd.DataFrame(columns=["cell_tower_id1","cell_tower_id2","source1_name","source2_name","device_name_1","device_name_2","frequency1","frequency2","frequency_band","latitude1","longitude1","latitude2","longitude2","district1","district2"])

['Source NE Name', 'Sink NE Name', 'Source NE Frequency (MHz)', 'Sink NE Frequency (MHz)', 'Band']
['Suntel ID', 'Site Name', 'Site Address', 'District', 'Province', 'Latitude-x', 'Longitude-x', 'Latitude', 'Longitude']
['DeviceID', 'ResourceName']


In [118]:
#Map the Frequency with locations
loc_device = pd.DataFrame(columns=["cell_tower_id1","cell_tower_id2","device_name_1","device_name_2","frequency1","frequency2","frequency_band",
                          "latitude1","longitude1","latitude2","longitude2","district1","district2"])
for index, row in link_frequencies_df.iterrows():
    dev_1_name = row["Source NE Name"]
    dev_2_name = row["Sink NE Name"]
    cell_tower_id1 = dev_1_name.split("-")[1].upper().strip()
    cell_tower_id2 = dev_2_name.split("-")[1].upper().strip()
    site_loc1 = sites_locations_df.loc[cell_tower_id1]
    site_loc2 = sites_locations_df.loc[cell_tower_id2]
    loc_device.loc[index] = [cell_tower_id1,cell_tower_id2,dev_1_name,dev_2_name,row["Source NE Frequency (MHz)"],
                             row["Sink NE Frequency (MHz)"],row["Band"],site_loc1["Latitude"],site_loc1["Longitude"],
                             site_loc2["Latitude"],site_loc2["Longitude"], site_loc1["District"],site_loc2["District"] ]
    
    

In [119]:
#Similar to joining in database tables, left Outer joining depending on the Device Name with the above computed loc_device and remove nulls
left_device_resources_df = device_resources_df.rename(columns={"ResourceName":"source1_name"})
leftMerge = left_device_resources_df.merge(loc_device,left_on="DeviceName",right_on="device_name_1",how="left")
npDropLeftMerge = leftMerge.dropna()
#     device2 = loc_device..loc[loc_device["device_name_2"]==index]

In [120]:
#Rejoin the device resources from above output to get the device 2 name set. Need to remove the number mismatiching links from this
right_device_resources_df = device_resources_df.rename(columns={"ResourceName":"source2_name"})
rightMerge = leftMerge.merge(right_device_resources_df,left_on="device_name_2",right_on="DeviceName",how="right")
npDropRightMerge = rightMerge.dropna()
names_re_orderd = npDropRightMerge[["source1_name","source2_name","cell_tower_id1","cell_tower_id2","device_name_1","device_name_2","frequency1","frequency2","frequency_band","latitude1","longitude1","latitude2","longitude2","district1","district2"]]

In [121]:
link_count=0
for index,row in names_re_orderd.iterrows():
    source1_type = row["source1_name"].split("-")[-4]
    source2_type = row["source2_name"].split("-")[-4]
    if(source1_type == source2_type):
        links.loc[link_count] = row
        link_count+=1

In [125]:
#Write the prepared links to a CSV file, There are some links that have been removed they are to be added by hand
links.to_csv(root_folder+outputFileName,index="false")